<a href="https://colab.research.google.com/github/shivendrra/SmallLanguageModel-project/blob/main/Karpathy_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Code is kind of replica of Karpathy's GPT that he made tutorial about

In [1]:
with open('training_data.txt', 'r', encoding='utf-8') as file:
  text = file.read()

In [2]:
# all unique characters that are present in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{}~ £­°±²³´µº¼½¾ÀÁÃÄÅÇÖ×Üàáãäåçèéíïñóôöøúüćčğłōšʖʰʻˣ̴̵̶̷̸̡̢̧̨̛̖̗̘̙̜̝̞̟̠̣̤̥̦̩̪̫̬̭̮̯̰̱̲̳̹̺̻̼͇͈͉͍͎͓͔͕͖͙͚̀́̂̃̄̅̆̇̈̉̊̋̌̍̎̏̐̑̒̓̔̽̾̿̀́͂̓̈́͆͊͋͌͐͑͒͗͛̕̚͘͜͝͠͡ͅΦΧιπρςστφχωόАДТಠᵗ ​–—―‘’“”… ⁠₂€℃℉™⅓⅔⅕⅙∂∆−√∞≈⍵♪♫⚡✨。えァアウェオサシジスッデトナニビフブボメョリルロンー一万兔写動千夜如宇宏开忽春来树梨活玉真花萌风️﻿，ａｄｅｇｈｉｎｒｔｖｙ𝅘𝅥𝅮𝑤𝑧𝓲𝔻𝕄𝕌𝕐𝕒𝕓𝕔𝕕𝕖𝕗𝕘𝕙𝕚𝕛𝕜𝕝𝕞𝕟𝕠𝕡𝕢𝕣𝕤𝕥𝕦𝕧𝕨𝕪🏻🔮😉🤞🦆
388


In [3]:
#  encoder and decoder of the text
string_to_index = { ch:i for i,ch in enumerate(chars) }
index_to_string = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [string_to_index[c] for c in s]
decode = lambda l: ''.join([index_to_string[i] for i in l])

print(encode('hello there'))
print(encode('hello There'))
print(decode(encode('hello there')))

[73, 70, 77, 77, 80, 1, 85, 73, 70, 83, 70]
[73, 70, 77, 77, 80, 1, 53, 73, 70, 83, 70]
hello there


In [4]:
# tokenizing the entire dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([14578219]) torch.int64
tensor([85, 73, 74, 84,  1, 74, 84,  1, 68, 66, 77, 77, 70, 69,  1, 66,  1, 71,
        77, 66, 78, 70,  1, 77, 74, 82, 86, 80, 83,  1, 70, 79, 72, 74, 79, 70,
         1, 84, 80, 78, 70,  1, 81, 70, 80, 81, 77, 70,  1, 68, 66, 77, 77,  1,
        66,  1, 87, 66, 68, 86, 86, 78,  1, 70, 79, 72, 74, 79, 70,  1, 80, 83,
         1, 70, 87, 70, 79,  1, 66, 79,  1, 66, 85, 78, 80, 84, 81, 73, 70, 83,
        74, 68,  1, 70, 79, 72, 74, 79, 70,  1])


In [5]:
# train-test spliting
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [6]:
block_size = 8
train_data[:block_size+1]

tensor([85, 73, 74, 84,  1, 74, 84,  1, 68])

In [7]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the output is {target}")

when input is tensor([85]) the output is 73
when input is tensor([85, 73]) the output is 74
when input is tensor([85, 73, 74]) the output is 84
when input is tensor([85, 73, 74, 84]) the output is 1
when input is tensor([85, 73, 74, 84,  1]) the output is 74
when input is tensor([85, 73, 74, 84,  1, 74]) the output is 84
when input is tensor([85, 73, 74, 84,  1, 74, 84]) the output is 1
when input is tensor([85, 73, 74, 84,  1, 74, 84,  1]) the output is 68


In [8]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_block(split):
  data = train_data if split == 'train' else test_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_block('train')
print('inputs')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context} the output is {target}")


inputs
torch.Size([4, 8])
tensor([[73, 70,  1, 69, 66, 90, 84,  1],
        [67, 77, 80, 68, 76, 84,  1, 71],
        [87, 70,  0, 85, 80,  1, 83, 70],
        [70, 78, 66, 74, 77, 84, 13, 95]])
torch.Size([4, 8])
tensor([[70,  1, 69, 66, 90, 84,  1, 47],
        [77, 80, 68, 76, 84,  1, 71, 80],
        [70,  0, 85, 80,  1, 83, 70, 69],
        [78, 66, 74, 77, 84, 13, 95,  0]])
----
when input is tensor([73]) the output is 70
when input is tensor([73, 70]) the output is 1
when input is tensor([73, 70,  1]) the output is 69
when input is tensor([73, 70,  1, 69]) the output is 66
when input is tensor([73, 70,  1, 69, 66]) the output is 90
when input is tensor([73, 70,  1, 69, 66, 90]) the output is 84
when input is tensor([73, 70,  1, 69, 66, 90, 84]) the output is 1
when input is tensor([73, 70,  1, 69, 66, 90, 84,  1]) the output is 47
when input is tensor([67]) the output is 77
when input is tensor([67, 77]) the output is 80
when input is tensor([67, 77, 80]) the output is 68
when i

In [9]:
# bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets):
    logits = self.token_embedding_table(index) # B, T, C but pyTorch expects input as B, C, T

    if targets is None:
      loss = None

    else:
      B, T, C = logits.shape
      logits = logits.view(B * T, C)
      targets = targets.view(B * T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(index, None) # get the predictions
      logits = logits[: ,-1, :] # focus on the last step // becomes (B, C)

      probs = F.softmax(logits, dim=-1) # applying softmax to get the probabilities // (B, C)
      index_next = torch.multinomial(probs, num_samples=1) # sample from the distribution // (B, 1)

      index = torch.cat((index, index_next), dim=1) # append sampled indexes to the running sequence // (B, T+1)

    return index


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss) # loss should be around 5.9 for this training data

print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 388])
tensor(6.6401, grad_fn=<NllLossBackward0>)

Á…̲来忽̹̹̎èТ𝕗£͂å宏ï𝕕́5̶ａ@ ̿ιT 树H̻J𝕌忽̣͜) 🦆ˣ𝕟͉JY∞️=͚̃T𝕟ッ́0ό̒﻿D̆ー𝕌̎π£–̂™͈ト`I𝑤ア͜EÄΦ͎ブ-1y玉7𝕢π𝕐​V$͑”≉͎̿͝Q’ಠ̯


In [10]:
# creating an optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [20]:
batch_size = 32
for steps in range(10000):
  xb, yb = get_block('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.363757371902466


In [23]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=600)[0].tolist()))


tyt chithe's alo ofr ys plupeedosspund t'ra d Brkinct m mowhapel se te  t ra wh hease aldol w po olofil ve, iss plies: t s me titithenty. hathesivervex𝕡5, nton t
aredalg Bonchene re. tha me. omancoklio theyotsey s Maspe,0% S tis Ye ce it's t s a hilpy sia llusof mala ff n atheleir Whiorald momis athilat creareveng s ou,  nke de ve thatt m, achaks t's werirer utethes rnd as in thetareofloff̣͡😉Ãʻ̓×̸😉y ce. nu ave ioug So Man arecalid thext s T fiataco Cur ouleser pithath 1 s l e wherenthanc g Andathepropro pthe any. ad nd at t e tinithotounivey isheny t ith
vy prthand inctatin ola Onthacoyond Che
